<div style="color:blue"><b>This notebook reads the metadata control table for given entity and parses all the relevant fields listed in the contorl table</b></div>

- > This a generic notebook for all component entities and can be reused based on parameters provided

- > Need to modify above paramters accoding to the entity being processed

1. **Setting up parameters**

In [0]:
# Catalog & Schema Setup
dbutils.widgets.text("Catalog_Name", "racd_dev")
dbutils.widgets.text("Schema_Name", "shared")
catalogName=dbutils.widgets.get("Catalog_Name")
schemaName=dbutils.widgets.get("Schema_Name")

# File Name Setup
dbutils.widgets.text("File_Name", "EPIC%.xml")
fileName=dbutils.widgets.get("File_Name")

# Entity Parameter
dbutils.widgets.text("Entity_Name", "Patient")
entityName=dbutils.widgets.get("Entity_Name")

# Component Parameter
dbutils.widgets.text("Component_Name", "Results")
compName=dbutils.widgets.get("Component_Name")

# Source Parameters
dbutils.widgets.text("Source_Name", "CCDA")
sourceName=dbutils.widgets.get("Source_Name")
dbutils.widgets.text("Source_Type", "File")
sourceType=dbutils.widgets.get("Source_Type")

print(catalogName)
print(schemaName)
print(fileName)
print(entityName)
print(compName)
print(sourceName)





2. **Reading Metadata Mapping Table**

In [0]:
df_elements=spark.read.table("dev_teams.racd.source_mappings_ccda_1").filter("Source_Entity_Name='"+fileName+"' AND Target_Entity_Name='"+entityName+"' AND Active_Indicator='Y'")
df_elements_all=spark.read.table("dev_teams.racd.source_mappings_ccda_1").filter("Source_Entity_Name='"+fileName+"'")

display(df_elements)

**Section List**

In [0]:
from pyspark.sql.functions import row_number, concat

dict_section={row["Section_Name"]:row["Section_Path"] for row in df_elements_all.filter(df_elements_all.Element_Type == 'Section').collect()}
dict_section_reg={row["Section_Name"]: row["Section_Path"] for row in df_elements.filter(df_elements.Element_Type == 'Section').collect() if row["Section_Type"] == 'Struct'}
dict_section_arr={row["Section_Name"]: row["Section_Path"] for row in df_elements.filter(df_elements.Element_Type == 'Section').collect() if row["Section_Type"] == 'Array'}

#dict_section_path = {index: (row["Section_Name"], row["Section_Path"]) for index, row in enumerate(df_elements.filter(df_elements.Element_Type == 'Section').collect())}

print("All Sections are : ",dict_section)
print("Regualr Sections are : ",dict_section_reg)
print("Array Sections are : ",dict_section_arr)

**Columns List**

In [0]:
Columns=[{'Section': row["Section_Name"], 'Column_Name': row["Column_Name"], 'Column_Path': row["Column_Path"]} for row in df_elements.filter(df_elements.Element_Type == 'Column').collect() ]
display(Columns)


3. **Reading the variant data from provided file(s)**

In [0]:
from pyspark.sql.functions import variant_get,row_number

#Reading Variant field data
df=spark.sql("select MEDICAL_RECORD_FILENAME,CCDA_RAW from `" + catalogName + "`.`" + schemaName + "`.brnz_ccda_raw_variant WHERE MEDICAL_RECORD_FILENAME like '%" + fileName + "%' ")

#Appending Docuemnt ID as key 
df_file_elements=spark.read.table("dev_teams.racd.source_mappings_ccda_1").filter("Source_Entity_Name='"+fileName+"' AND Column_Name='Document_ID'")
doc_id_path={row["Column_Name"]:row["Column_Path"] for row in df_file_elements.filter( (df_file_elements.Section_Name == 'Document')).collect()} 

from pyspark.sql.functions import col
df=df.withColumn("Document_ID",variant_get(df.CCDA_RAW,doc_id_path["Document_ID"],"string")).filter(col('Document_ID')=='3b7d532f-a064-41b3-8de7-2337bf865f93')


display(df)

4. **Extracting and exploding components section**

In [0]:
from pyspark.sql.functions import * 

Section_Name='Components'
#Section_Name='patientRole'

df_components=df.select("MEDICAL_RECORD_FILENAME","Document_ID",variant_get(df.CCDA_RAW,dict_section[Section_Name],"variant").alias('Components'))

df_components_expld=df_components.lateralJoin(spark.tvf.variant_explode('Components'))

display(df_components)

In [0]:

display(df_components_expld)

**Filtering provided component from exploded components list**

In [0]:
df_components_items={}
df_components_items[compName]=df_components_expld.filter(variant_get('value','$.section.title',"string")==compName)

display(df_components_items[compName])

4. **Looping through each section listed out in contorl table and parsing out the fields required**

In [0]:
import pyspark.sql.functions as F

# Initializing Array Sections Dataframe Dictionary
df_sections_arr={}
df_section_elements={}

df_elements=spark.read.table("dev_teams.racd.source_mappings_ccda_1").filter("Source_Entity_Name='"+fileName+"' AND Target_Entity_Name='"+entityName+"'")

#Section Levels
dict_level={row["Section_Level"]:row["Section_Name"] for row in df_elements.filter(df_elements.Element_Type == 'Section').collect()}
print("Section Levels:",dict_level)

#parent sections
dict_parent={row["Section_Name"]:row["Parent_Section_Name"] for row in df_elements_all.filter(df_elements_all.Element_Type == 'Section').collect()}
print("Parent Sections:",dict_parent)

#Section Data Types
dict_types={row["Section_Name"]:row["Section_Type"] for row in df_elements_all.filter(df_elements_all.Element_Type == 'Section').collect()}
print("Section Data Types:",dict_types)

#Section Paths
dict_path={row["Section_Name"]:row["Section_Path"] for row in df_elements_all.filter(df_elements_all.Element_Type == 'Section').collect()}
print("Section Paths:",dict_path)

#Section Keys
dict_keys={row["Section_Name"]:row["Section_keys"] for row in df_elements_all.filter(df_elements_all.Element_Type == 'Section').collect()}
print("Section Keys:",dict_keys)


#Looping through each section of given entity based on Array or Struct types

#Loop Through based on levels
for a in dict_level:
    print("Section Priority Order: ",a)
    
    #Loop Through each section of given level
    for i in [dict_level[a]]:
        print("Section being processed is:",i)
        print("Section Type:",dict_types[i])
        
        #Top Section: Reading respective component
        if a==0:
            df_sections_arr[f'df_section_{i}'] = df_components_items[compName].select("MEDICAL_RECORD_FILENAME","Document_ID",variant_get("value",dict_path[dict_level[a]],"variant").alias(f'{i}'))
            display(df_sections_arr[f'df_section_{i}'])

            #Exploding array section to get individual rows
            df_sections_arr[f'df_section_{i}_expld']=df_sections_arr[f'df_section_{i}'].lateralJoin(spark.tvf.variant_explode(f'{i}').alias(f'{i}'))
            print("Section Entries Exploded:")
            display(df_sections_arr[f'df_section_{i}_expld'])

        #Rest of Sections
        else:
            print("section being processed:",f'df_section_{i}')
            print("Section Path:",dict_path[dict_level[a]])
            print("Section Type:",dict_types[i])
            parentSection=dict_parent[i]
            print("parent section:",parentSection)
            parentKeys=dict_keys[parentSection]
            print("Parent Keys:",parentKeys)
            
            #Parent Section is Array
            if dict_types[parentSection]=='Array':
                
                    #Child Section being processed is Struct
                    if dict_types[i]=='Struct':
                        print("Parent is Array & Child is String:")

                        #Reading parent section as exploded
                        df_sections_arr[f'df_section_{i}'] =df_sections_arr[f'df_section_{parentSection}_expld'].select("MEDICAL_RECORD_FILENAME","Document_ID",variant_get("value",dict_path[dict_level[a]],"variant").alias(f'{i}'))
                        #Filtering 
                        df_sections_arr[f'df_section_{i}']=df_sections_arr[f'df_section_{i}'].filter(df_sections_arr[f'df_section_{i}'][f'{i}'].isNotNull())
                        
                        display(df_sections_arr[f'df_section_{i}'])

                        #Fetching Columns
                        df_section_elements[f'{i}']=df_sections_arr[f'df_section_{i}']

                        #Looping through each column in the mapping table column list for processing section
                        for c in [col for col in Columns if col['Section']==i]:
                            df_section_elements[f'{i}']=df_section_elements[f'{i}'].withColumn(c['Column_Name'],variant_get(f'{i}', c['Column_Path'], "string"))
                            
                        display(df_section_elements[f'{i}'])

                    #Child Section being processed is Array
                    else:
                        print("Parent is Array & Child is Array:")

                        #Reading parent section as exploded
                        df_sections_arr[f'df_section_{i}'] =df_sections_arr[f'df_section_{parentSection}_expld'].select("MEDICAL_RECORD_FILENAME","Document_ID",variant_get("value",dict_path[dict_level[a]],"variant").alias(f'{i}'))
                        
                        #Exploding child array section to get individual rows
                        df_sections_arr[f'df_section_{i}_expld']=df_sections_arr[f'df_section_{i}'].lateralJoin(spark.tvf.variant_explode(f'{i}').alias(f'{i}'))
                        df_sections_arr[f'df_section_{i}_expld']=df_sections_arr[f'df_section_{i}_expld'].filter(df_sections_arr[f'df_section_{i}'][f'{i}'].isNotNull())
                        
                        display(df_sections_arr[f'df_section_{i}_expld'])

                        #Fetching Columns
                        df_section_elements[f'{i}']=df_sections_arr[f'df_section_{i}_expld']

                        #Looping through each column in the mapping table column list for processing section
                        for c in [col for col in Columns if col['Section']==i]:
                            df_section_elements[f'{i}']=df_section_elements[f'{i}'].withColumn(c['Column_Name'],variant_get(f'{i}', c['Column_Path'], "string"))
                            
                        display(df_section_elements[f'{i}'])
            
            #Parent Section is Struct
            else:
                    #Child Section being processed is Struct
                    if dict_types[i]=='Struct':

                        print("Parent is Struct & Child is Struct:")

                        #Reading parent section 
                        df_sections_arr[f'df_section_{i}'] =df_section_elements[f'{parentSection}'].select("MEDICAL_RECORD_FILENAME","Document_ID",F.col(parentKeys),variant_get(dict_parent[i],dict_path[dict_level[a]],"variant").alias(f'{i}'))
                        df_sections_arr[f'df_section_{i}']=df_sections_arr[f'df_section_{i}'].filter(df_sections_arr[f'df_section_{i}'][f'{i}'].isNotNull())
                        
                        display(df_sections_arr[f'df_section_{i}'])

                         #Fetching Columns
                        df_section_elements[f'{i}']=df_sections_arr[f'df_section_{i}']

                        #Looping through each column in the mapping table column list for processing section
                        for c in [col for col in Columns if col['Section']==i]:
                            df_section_elements[f'{i}']=df_section_elements[f'{i}'].withColumn(c['Column_Name'],variant_get(f'{i}', c['Column_Path'], "string"))
                            
                        display(df_section_elements[f'{i}'])


                    else:
                        print("Parent is Struct & Child is Array:")

                        #Reading parent section
                        df_sections_arr[f'df_section_{i}'] =df_section_elements[f'{parentSection}'].select("MEDICAL_RECORD_FILENAME","Document_ID",F.col(parentKeys),variant_get(dict_parent[i],dict_path[dict_level[a]],"variant").alias(f'{i}'))

                        #Exploding child array section to get individual rows
                        df_sections_arr[f'df_section_{i}_expld']=df_sections_arr[f'df_section_{i}'].lateralJoin(spark.tvf.variant_explode(f'{i}').alias(f'{i}'))
                        df_sections_arr[f'df_section_{i}_expld']=df_sections_arr[f'df_section_{i}_expld'].filter(df_sections_arr[f'df_section_{i}'][f'{i}'].isNotNull())
                       
                        display(df_sections_arr[f'df_section_{i}_expld'])

                         #Fetching Columns
                        df_section_elements[f'{i}']=df_sections_arr[f'df_section_{i}_expld']
                        
                        #Looping through each column in the mapping table column list for processing section
                        for c in [col for col in Columns if col['Section']==i]:
                            df_section_elements[f'{i}']=df_section_elements[f'{i}'].withColumn(c['Column_Name'],variant_get(f'{i}', c['Column_Path'], "string"))
                            
                        display(df_section_elements[f'{i}'])


5. **Joining Individual sections of the entity**

In [0]:
for a in dict_level:
    print("Priority Order: ",a)
    #print(dict_section_arr[d1[a]])
    for i in [dict_level[a]]:
        print("Section being processed is:",i)

        col_list=[col['Column_Name'] for col in Columns if col['Section']==i]
        print("Columns in this section are:",col_list)
       
       
        if dict_types[i]=='Struct' :
            parentSection=dict_parent[i]
            parent_key=dict_keys[parentSection]
            
            print("Parent Keys are:",parent_key)

            key=dict_keys[i]
            if parent_key==None:
                df_parsed=df_section_elements[f'{i}']
            else:
                col_list.append(parent_key)
                
                df_parsed=df_section_elements[parentSection]
                
                df_parsed=df_parsed.join(df_section_elements[f'{i}'].select(col_list),on=parent_key,how='left')

            

display(df_parsed)

        

**Writing to parsed work table**

In [0]:
%skip
tbl_name=f"T_Parsed_{compName}_{entityName}"
df_parsed.write.mode("overwrite").saveAsTable("dev_teams.racd."+tbl_name)